In [63]:
import pandas as pd
import numpy as np
import os
from Bio import AlignIO


In [64]:
data = os.listdir(os.getcwd())

#dfs = [pd.read_csv(i) for i in data if ".csv" in i]
data

['.ipynb_checkpoints',
 'antigens_supp_dat.xlsx',
 'indels_filter_summary.csv',
 'indels_passes_all.csv',
 'isovar_indels_results.csv',
 'Isovar_results_summary.ipynb',
 'isovar_snvs_results.csv',
 'snvs_filter_summary.csv',
 'snvs_passes_all.csv']

In [10]:
indels = dfs[0]
snvs = dfs[1]

In [43]:
def summarize_isovar(df):
    filt = df[[i for i in df.columns if "filter" in i]]
    new_df = pd.DataFrame()
    for col in filt.columns:
        new_df[col] = filt[col].value_counts()
        
    new_df = new_df.T.fillna(0)
    new_df["percent_true"] = np.round((new_df[True] / (new_df[True] + new_df[False])) * 100, 2)
    
    return new_df

summarize_isovar(snvs).to_csv("snvs_filter_summary.csv")

In [48]:
def batch_read_csv(file_paths, keyword):
    return [pd.read_csv(i) for i in file_paths if keyword in i]

summaries = batch_read_csv(data, "summary")
passes_all = batch_read_csv(data, "passes_all")

In [98]:
s_pass = pd.read_csv("snvs_passes_all.csv", index_col="Unnamed: 0")
i_pass = pd.read_csv("indels_passes_all.csv", index_col="Unnamed: 0")
anti = pd.read_excel("antigens_supp_dat.xlsx")
s_all = pd.read_csv("isovar_snvs_results.csv")

In [66]:
bbn963 = anti.groupby("Tumor_model").get_group("BBN963")

In [110]:
s_all_dropped = s_all.loc[s_all_dropped.predicted_effect_mutant_protein_sequence.dropna().index]

In [142]:
def match_seq(df, antis, col):
    dic = {}
    idx = []
    df = pd.DataFrame()
    for a in antis:
        for i in df.index:
            if a in df[col].loc[i]:
                dic[a] = (i, df[col].loc[i])
                idx.append(i)
                
    return df.loc[set(idx)], dic
            
            

match_rna, rna_dic = match_seq(s_pass, bbn963.Antigen, "protein_sequence")
match_dna, dna_dic = match_seq(s_pass, bbn963.Antigen, "predicted_effect_mutant_protein_sequence")
#all_dict = match_seq(s_all_dropped, bbn963.Antigen, "predicted_effect_mutant_protein_sequence")

In [191]:
def summarize_matches(df, antis):

    summ_df = pd.DataFrame(index=antis.values, columns=["match_index", "protein_sequence", "predicted_effect_mutant_protein_sequence", "Both", "passes_all_filters"])
    
    for a in antis:
        for i in df.index:
            
            try:
                if a in df.loc[i, 'protein_sequence']:
                    summ_df.loc[a, "passes_all_filters"] = df.loc[i, 'passes_all_filters']
                    summ_df.loc[a, "match_index"] = i
                    summ_df.loc[a, 'protein_sequence'] = True
            except TypeError:
                pass
            
            try:
                if a in df.loc[i, "predicted_effect_mutant_protein_sequence"]:
                    summ_df.loc[a, "passes_all_filters"] = df.loc[i, 'passes_all_filters']
                    summ_df.loc[a, "match_index"] = i
                    summ_df.loc[a, 'predicted_effect_mutant_protein_sequence'] = True
            except TypeError:
                pass
    for i in summ_df.index:
        if summ_df.loc[i, 'protein_sequence'] != True:
            summ_df.loc[i, ['protein_sequence', 'Both']] = False
            
        if summ_df.loc[i, 'predicted_effect_mutant_protein_sequence'] != True:
            summ_df.loc[i, ['predicted_effect_mutant_protein_sequence', 'Both']] = False
    
        
    summ_df.Both = summ_df.Both.fillna(True)
    
    return summ_df

summ = summarize_matches(s_all, bbn963.Antigen)

In [243]:
matches = summ.dropna().drop_duplicates("match_index")


#matches.to_csv("isovar_paper_matches.csv")

counts = pd.DataFrame(matches.iloc[:, 1:].astype(bool).sum(), columns=[True])#, False, "percent"])
counts[False] = 14 - counts


matches
#matches.reset_index().rename(columns={"index":"Antigen"}).to_csv("isovar_paper_matches.csv", index=False)

,match_index,protein_sequence,predicted_effect_mutant_protein_sequence,Both,passes_all_filters
FSLNVVNL,7292,True,True,True,True
FTFTSFWAYKI,9926,True,True,True,True
GMAVRALNM,4530,True,True,True,True
IAICVSDTV,18133,True,True,True,True
ISRNRHTL,16309,True,True,True,True
SMGSNDSDTL,12136,False,True,False,False
YFLENLDKL,3791,True,True,True,True
ETLLNSATI,2683,True,True,True,True
LMMLKNVGFAL,14271,True,True,True,True
SNVMQLLL,9800,True,True,True,False


In [134]:
bbn963.loc[bbn963.Antigen.isin(list(s_dict.keys()))]

,Antigen,Reference,Predicted_MHC_class,Tumor_model,MHC_haplotype,ELISpot_score,Group,GBM_score
19,FSLNVVNL,FSPNVVNL,I,BBN963,H2-Kb,-51.333333,Exploration,1.933884
20,FSLNVVNLTLV,FSPNVVNLTLV,I,BBN963,H2-Db,283.333333,Exploration,39.644072
21,FTFTSFWAYKI,FIFTSFWAYKI,I,BBN963,H2-Kb,-23.166700,Exploration,34.773693
24,GMAVRALNM,GMAVRELNM,I,BBN963,H2-Kb,23.833333,Exploration,34.773693
35,IAICVSDTV,IAICVSVTV,I,BBN963,H2-Db,33.000000,Exploration,43.318294
38,ISRNRHTL,ISCNRHTL,I,BBN963,H2-Kb,13.833332,Exploration,1.933884
125,VMISRNRHTL,VMISCNRHTL,I,BBN963,H2-Kb,12.000000,Exploration,37.532123
130,VVMISRNRHTL,VVMISCNRHTL,I,BBN963,H2-Kb,-7.833330,Exploration,37.532123
136,YFLENLDKL,YFLDNLDKL,I,BBN963,H2-Db,-18.166700,Exploration,1.933884
148,ETLLNSATI,ETLLNSATT,I,BBN963,H2-Db,-41.333333,Validation,1.933884


In [49]:
s_pass.protein_sequence.sort_values().iloc[0:10]

568    AAELFGLTAEKIYLVHDELD
299    AAREAAKTVKTKCGSKLFPL
728          AASGSSASSIHFSS
439    AASQRRDVERELTRFMAKTG
901    AATSNTTSCSNPVAASKMST
745    ACEMVADMVETLQSVLALGH
938        ADKTGDIIVTNDNFRE
384    ADLLPPPPAQPPPHSNSEEY
197    AEAGYTSRGKSGCTQPRRVA
386    AEEHQRVSKGNLKAFGNSTE
Name: protein_sequence, dtype: object

In [37]:
s_pass.

Index(['variant', 'overlapping_gene_names', 'overlapping_gene_ids',
       'overlapping_coding_gene_names', 'overlapping_coding_gene_ids',
       'fraction_alt_fragments', 'fraction_alt_reads',
       'fraction_other_fragments', 'fraction_other_reads',
       'fraction_ref_fragments', 'fraction_ref_reads', 'num_alt_fragments',
       'num_alt_reads', 'num_amino_acid_mismatches_from_predicted_effect',
       'num_amino_acid_mismatches_from_reference',
       'num_cdna_mismatches_after_variant_in_top_protein_sequence',
       'num_cdna_mismatches_before_variant_in_top_protein_sequence',
       'num_cdna_mismatches_in_top_protein_sequence',
       'num_cdna_sequences_from_top_protein_sequence',
       'num_fragments_supporting_top_protein_sequence',
       'num_genes_from_protein_sequences',
       'num_genes_from_top_protein_sequence',
       'num_mutant_amino_acids_in_protein_sequence', 'num_other_fragments',
       'num_other_reads', 'num_overlapping_coding_genes',
       'num_overlapp